In [1]:
import numpy as np
import pandas as pd
import main
fake_data_wc = pd.read_csv('fake_data/fake_wc_res.csv', delimiter=';')
fake_data_wc.head()

,ID,X,Y,WC
0,0,348202,6460000,1.700000e+38
1,1,348202,6460000,1.700000e+38
2,2,348202,6460000,1.700000e+38
3,3,348202,6460000,1.700000e+38
4,4,348202,6460000,1.700000e+38


In [2]:
# Конвертируем форматы
fake_data_wc['WC'] = fake_data_wc['WC'].astype('float64')
fake_data_wc['ID'] = fake_data_wc['ID'].astype('str')
fake_data_wc['WELL_NAME'] = 'fake_' + fake_data_wc['ID']
del fake_data_wc['ID']
fake_data_wc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5399 entries, 0 to 5398
Data columns (total 4 columns):
X            5399 non-null int64
Y            5399 non-null int64
WC           5399 non-null float64
WELL_NAME    5399 non-null object
dtypes: float64(1), int64(2), object(1)
memory usage: 168.8+ KB


In [3]:
# Удаляем значения зашедшие за край
fake_data_wc = fake_data_wc.drop(fake_data_wc[fake_data_wc['WC'] > 1].index)
fake_data_wc.head()

,X,Y,WC,WELL_NAME
24,348302,6460000,0.39,fake_24
25,348302,6460000,0.39,fake_25
26,348302,6460000,0.39,fake_26
27,348302,6460000,0.39,fake_27
28,348302,6460000,0.39,fake_28


In [4]:
fake_las_out = pd.read_csv('fake_data/fake_las_out.csv', delimiter=';')
fake_las_out.head()

,WELL_NAME,DEPTH,APS
0,fake_0,2584.999314,0.380130
1,fake_0,2585.121187,0.380130
2,fake_0,2585.121309,0.364062
3,fake_0,2585.243182,0.364062
4,fake_0,2585.243304,0.309785


In [5]:
fake_bottom = pd.read_csv('fake_data/fake_bottom_res.csv', delimiter=';')
fake_bottom['ID'] = fake_bottom['ID'].astype('str')
fake_bottom['WELL_NAME'] = 'fake_' + fake_bottom['ID']
fake_bottom = fake_bottom.rename(columns={'DEPTH': 'DEPTH_BOTTOM'})
del fake_bottom['ID']
del fake_bottom['X']
del fake_bottom['Y']
fake_bottom.head()

,DEPTH_BOTTOM,WELL_NAME
0,2601.50,fake_0
1,2600.09,fake_1
2,2598.85,fake_2
3,2598.79,fake_3
4,2599.31,fake_4


In [6]:
fake_top = pd.read_csv('fake_data/fake_top_res.csv', delimiter=';')
fake_top['ID'] = fake_top['ID'].astype('str')
fake_top['WELL_NAME'] = 'fake_' + fake_top['ID']
fake_top = fake_top.rename(columns={'DEPTH': 'DEPTH_TOP'})
del fake_top['ID']
del fake_top['X']
del fake_top['Y']
fake_top.head()

,DEPTH_TOP,WELL_NAME
0,2585.02,fake_0
1,2582.98,fake_1
2,2581.22,fake_2
3,2580.52,fake_3
4,2580.87,fake_4


In [7]:
fake_data = pd.merge(fake_data_wc, fake_las_out, on='WELL_NAME')
fake_data = pd.merge(fake_data, fake_top, on='WELL_NAME') 
fake_data = pd.merge(fake_data, fake_bottom, on='WELL_NAME')
fake_data.head()

,X,Y,WC,WELL_NAME,DEPTH,APS,DEPTH_TOP,DEPTH_BOTTOM
0,348302,6460000,0.39,fake_24,2588.656453,0.274434,2588.27,2603.89
1,348302,6460000,0.39,fake_24,2588.771455,0.274434,2588.27,2603.89
2,348302,6460000,0.39,fake_24,2588.771570,0.274434,2588.27,2603.89
3,348302,6460000,0.39,fake_24,2588.886333,0.191055,2588.27,2603.89
4,348302,6460000,0.39,fake_24,2588.886448,0.101674,2588.27,2603.89


In [8]:
from scipy import interpolate
well_name_list = fake_data['WELL_NAME'].value_counts().index.tolist()
x_values = []
y_values = []
count_val = 100
well_count = 0
for well_name in well_name_list:
    if well_count%500 == 0:
        print(well_count, ' of ', len(well_name_list))
    data_well =  fake_data[fake_data['WELL_NAME'] == well_name]
    bottom = data_well['DEPTH_BOTTOM'].tolist()[0]
    top = data_well['DEPTH_TOP'].tolist()[0]
    # print('top = ', top)
    # print('bottom =', bottom)    
    data_well_by_bound = data_well[(data_well['DEPTH'] >= top) & (data_well['DEPTH'] <= bottom)]
    x_arr = data_well_by_bound['DEPTH']
    y_arr = data_well_by_bound['APS']
    
    # print ('length of array depth', len(x_arr))
    f_spline = interpolate.interp1d(x_arr, y_arr, kind = 'slinear')
    h_start = data_well_by_bound['DEPTH'].min()
    h_end = data_well_by_bound['DEPTH'].max()
    # print(h_start, h_end, top, bottom)
    h_step = (h_end - h_start)/count_val
    x_temp = []
    i = 0
    while(i < count_val):
        x_temp.append(float(f_spline(h_start + i * h_step)))
        i = i + 1
    x_values.append(x_temp)  
    y_values.append(data_well['WC'].tolist()[0])
    well_count = well_count + 1
print('end fill')

0  of  5322
500  of  5322
1000  of  5322
1500  of  5322
2000  of  5322
2500  of  5322
3000  of  5322
3500  of  5322
4000  of  5322
4500  of  5322
5000  of  5322
end fill


In [9]:
data_well_for_analize = pd.DataFrame({'aps':x_values, 'wc':y_values})
data_well_for_analize.to_csv("fake_data\\fake_data_collect.csv", sep = ';')

In [10]:
y_values_r = [int(10 * y_val) for y_val in y_values]

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

x_train, x_hold, y_train, y_hold = train_test_split(x_values, y_values_r, test_size=0.3, random_state=17)

In [14]:
tree = DecisionTreeClassifier(max_depth=20, random_state=17, criterion='gini')
tree.fit(x_train, y_train)
tree_pred = tree.predict(x_hold)
accuracy_score(y_hold, tree_pred)

0.70569818409517848

In [15]:
len(y_train)

3725

In [24]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
# вычисляем минимум и максимум по обучающим данным
scaler = MinMaxScaler(). fit(x_train)
# масштабируем обучающие данные
print('x_traind: ', np.min(x_train), np.max(x_train))
print('x_hold:', np.min(x_hold), np.max(x_hold))
x_train_scaled = scaler.transform(x_train)
x_hold_scaled = scaler.transform(x_hold)


x_traind:  -0.483299255 1.2203967571
x_hold: -0.365327686 1.16451554852


In [26]:
# обучаем дерево
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

forest = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=17)
# print(np.mean(cross_val_score(forest, x_hold, y_hold, cv=6))) # 0.949


forest_params = { 
    'max_depth': [1, 10, 20, 40, 60],
    'n_estimators': [1, 10, 30, 50, 100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2']
}

forest_grid = GridSearchCV(forest, forest_params, cv=5, n_jobs=-1, verbose=True)
 
forest_grid.fit(x_train_scaled, y_train)

Fitting 5 folds for each of 105 candidates, totalling 525 fits


D:\libs\Anaconda\3.3\envs\aind\lib\site-packages\sklearn\model_selection\_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 525 out of 525 | elapsed:  7.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=17,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [1, 10, 20, 40, 60], 'n_estimators': [1, 10, 30, 50, 100, 200, 500], 'max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [27]:
print('forest_grid.best_params_ = ', forest_grid.best_params_)
print('forest_grid.best_score_ = ', forest_grid.best_score_)
print('forest_grid.best_estimator_ = ', forest_grid.best_estimator_)
print('accuracy_score: ', accuracy_score(y_hold, forest_grid.predict(x_hold_scaled)))

forest_grid.best_params_ =  {'max_depth': 40, 'max_features': 'log2', 'n_estimators': 500}
forest_grid.best_score_ =  0.818791946309
forest_grid.best_estimator_ =  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=40, max_features='log2', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=-1, oob_score=False, random_state=17,
            verbose=0, warm_start=False)
accuracy_score:  0.837194740138


In [19]:
y_calc_val = forest_grid.predict(x_hold)

In [105]:
y_uniq = []
for y in  y_calc_val:
    is_uniq = True
    for y_un in y_uniq:
        if (y_un == y):
            is_uniq = False
    if (is_uniq):
        y_uniq.append(y)
y_uniq

[3, 4, 2, 1, 5]

In [71]:
import pickle
# Dump the trained decision tree classifier with Pickle
decision_tree_pkl_filename = 'fake_data\\decision_forest_grid_fake_data.pkl'
# Open the file to save as pkl file
decision_tree_model_pkl = open(decision_tree_pkl_filename, 'wb')
pickle.dump(forest_grid, decision_tree_model_pkl)
# Close the pickle instances
decision_tree_model_pkl.close()

In [79]:
import pickle
# Dump the trained decision tree classifier with Pickle
decision_tree_pkl_filename = 'fake_data\\decision_tree_classifier_fake_data.pkl'
# Open the file to save as pkl file
decision_tree_model_pkl = open(decision_tree_pkl_filename, 'wb')
pickle.dump(tree, decision_tree_model_pkl)
# Close the pickle instances
decision_tree_model_pkl.close()